# Цепочка обязанностей (chain of responsibility)

Цепочка обязанностей — это поведенческий паттерн проектирования, который позволяет передавать запросы последовательно по цепочке обработчиков. Каждый последующий обработчик решает, может ли он обработать запрос сам и стоит ли передавать запрос дальше по цепи.

## Пример

Вы автоматизируете службу техническое поддержки. В организации всего 3 уровня тех. поддержки (от первого до третьего). Чем выше уровень, тем более сложные проблемы может решать специалист. При возникновении проблемы, клиент оформляет заявку и отправляет ее на обработку. Требуется реализовать алгоритм распределения заявок по специалистам тех. поддержки.

In [35]:
from __future__ import annotations
import abc
from typing import Optional
from dataclasses import dataclass


@dataclass
class Request:
    """
    Запрос от клиента
    """

    technical_support_line: str
    details_message: str


class AbsHandler(abc.ABC):
    """
    Абстрактный класс обработчика с частичной реализацией.
    """

    _handler: Optional[AbsHandler] = None

    def set_next(self, handler: AbsHandler) -> AbsHandler:
        """
        Добавляет следующий обработчик.
        """
        self._handler = handler
        return handler

    @abc.abstractmethod
    def handle(self, request: Request) -> None:
        """
        Метод обработки запроса.
        """
        if self._handler is None:
            return
        # Передаем запрос следующему обработчику
        self._handler.handle(request)


class FirstLine(AbsHandler):
    def handle(self, request: Request) -> None:
        if request.technical_support_line == "first_line":
            print(
                "[1-линия тех. поддержки] Попробуйте перезапустить и прочесть инструкцию"
            )
        else:
            super().handle(request)


class SecondLine(AbsHandler):
    def handle(self, request: Request) -> None:
        if request.technical_support_line == "second_line":
            print(
                "[2-линия тех. поддержки] Посмотрите на световой индикатор на передней панели"
            )
        else:
            super().handle(request)


class ThirdLine(AbsHandler):
    def handle(self, request: Request) -> None:
        if request.technical_support_line == "third_line":
            print(
                "[3-линия тех. поддержки] Отройте отладчик и передайте содержимое стека"
            )
        else:
            super().handle(request)

In [37]:
requests = [
    Request(
        technical_support_line="first_line",
        details_message="Подключил к сети TP-Link, а он не раздает сеть по WI-FI",
    ),
    Request(
        technical_support_line="third_line",
        details_message="На свежих драйверах Nvidia в безопасном режиме не работает сглаживание",
    ),
]

handler = FirstLine()

# Выстраиваем все обработчики в цепочку
prev_handler = handler
for current_handler in (SecondLine(), ThirdLine()):
    prev_handler = prev_handler.set_next(current_handler)


# Для каждого запроса, достаточно вызвать обработку у первого обработчика в цепи
for request in requests:
    print("[запрос клиента]:", request.details_message)
    handler.handle(request)
    print()

[запрос клиента]: Подключил к сети TP-Link, а он не раздает сеть по WI-FI
[1-линия тех. поддержки] Попробуйте перезапустить и прочесть инструкцию

[запрос клиента]: На свежих драйверах Nvidia в безопасном режиме не работает сглаживание
[3-линия тех. поддержки] Отройте отладчик и передайте содержимое стека

